# Kickstarter project

In [0]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## Only run if on Google Colab

In [67]:
#from google.colab import drive
#drive.mount('/content/drive')

#from google.colab import files
#files.upload()

{}

## Try 2.0 -> Using github clone, might be faster

In [68]:
!git clone https://github.com/Strongkong/cleaned_kickstarted_dataset

fatal: destination path 'cleaned_kickstarted_dataset' already exists and is not an empty directory.


In [0]:
folder = "./cleaned_kickstarted_dataset/"
filename = "test_joined.csv"
fullpath = folder + filename

## Load csv file

In [0]:
df = pd.read_csv(fullpath, encoding='utf-8', sep=',')

## Remove whitespaces from header names

In [0]:
df = df.rename(columns=lambda x: x.strip())


## Peak at head

In [72]:
print (df.blurb[3])

Für die Finanzierung der Veröffentlichung meiner Masterarbeit, die dann als eigenständiges Buch bei Turia&Kant erscheint.


In [73]:
df.head()

,blurb,spotlight,staff_pick,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,If you've ever been disheartened by how some p...,True,False,62545913,"""How We've Changed Jesus"" - Neal Samudre's Deb...",Nonfiction,Publishing,USD,2013-04-12,3500.0,2013-04-04 22:46:00,3651.0,successful,80,US,3651.00,3651.00,3500.00
1,This book will show you how to attract your id...,True,False,2031730466,Your Client Vision (Book) - How to sell withou...,Nonfiction,Publishing,GBP,2017-01-16,175.0,2016-12-18 20:09:14,271.0,successful,31,GB,94.80,332.73,214.86
2,I told my first lie when I was six and never s...,True,False,614137516,The Longest Lie I Ever Told,Nonfiction,Publishing,CAD,2016-11-08,3000.0,2016-10-09 18:32:56,3600.0,successful,74,CA,440.30,2668.45,2223.70
3,Für die Finanzierung der Veröffentlichung mein...,True,False,1164271753,Spurenlesen – Zur Philosophie der Human-Animal...,Nonfiction,Publishing,EUR,2016-08-14,1000.0,2016-07-15 14:17:36,1080.0,successful,12,DE,222.27,1223.21,1132.61
4,Six teens (age 13-15) are hiking VT's 272-mile...,True,False,384486589,Teens Hike VT's Long Trail and Write How-To an...,Nonfiction,Publishing,USD,2014-06-01,3700.0,2014-04-25 02:24:24,4051.2,successful,44,US,4051.20,4051.20,3700.00


## Main categorical features

### Categories

In [74]:
categories = df.category.unique()
print(categories)

['Nonfiction' 'Publishing' 'Mixed Media' 'Web' 'People' 'Comic Books'
 'Comedy' 'Punk' 'Technology' 'Theater' 'Jazz' 'Software' 'Anthologies'
 'Science Fiction' 'Illustration' 'Gadgets' 'Product Design' 'Farms'
 'Workshops' 'Footwear' 'Weaving' 'Sculpture' 'Experimental' 'Pet Fashion'
 'Art' 'Graphic Design' 'Civic Design' 'Tabletop Games' 'Calendars'
 'Design' 'Electronic Music' 'Installations' 'Family' 'Festivals'
 'Conceptual Art' 'Performance Art' 'Glass' 'Painting' 'Plays' 'Jewelry'
 'Letterpress' 'Public Art' 'Accessories' 'DIY' 'Small Batch'
 'Video Games' 'Fantasy' 'Music' 'Comics' 'Digital Art' 'Immersive'
 'Hardware' 'Radio & Podcasts' "Children's Books" 'Flight' 'Fine Art'
 'Musical' 'Rock' 'Apparel' 'Nature' 'Games' 'Crafts' 'Robots'
 'Stationery' 'DIY Electronics' 'Space Exploration' 'Apps'
 'Movie Theaters' 'Woodworking' 'Printing' 'Knitting' 'Crochet'
 'Camera Equipment' 'Wearables' 'Makerspaces' 'Pottery' 'Photography'
 'Fashion' 'Candles' 'Embroidery' 'Photobooks' 'Art

### Main categories

In [75]:
main_categories = df.main_category.unique()
print(main_categories)

['Publishing' 'Art' 'Journalism' 'Photography' 'Comics' 'Film & Video'
 'Music' 'Technology' 'Theater' 'Design' 'Food' 'Dance' 'Fashion' 'Crafts'
 'Games']


### Currencies

In [76]:
currencies = df.currency.unique()
print(currencies)

['USD' 'GBP' 'CAD' 'EUR' 'AUD' 'NZD' 'SEK' 'NOK' 'DKK' 'MXN' 'HKD' 'SGD'
 'CHF' 'JPY']


### States

In [77]:
states = df.state.unique()
print(states)

['successful' 'live' 'failed' 'canceled' 'suspended']


### Country

In [78]:
countries = df.country.unique()
print(countries)

['US' 'GB' 'CA' 'DE' 'AU' 'BE' 'FR' 'ES' 'NZ' 'SE' 'LU' 'NL' 'NO' 'AT'
 'DK' 'MX' 'IE' 'IT' 'HK' 'SG' 'CH' 'JP']


## Data visualization

### Categorical features

In [0]:
categorical_features = ['category', 'main_category', 'state', 'country', 'currency']
from copy import deepcopy
data = deepcopy(df)
n_items = len(data)

### COMMENTED OUT Only display common ones (source: laboratory 1)

index_ = 1

plt.subplots_adjust(hspace=3.5)
for feat in categorical_features:
  fig = plt.figure(figsize=(11,12), dpi=100)
  ax = fig.add_subplot(1,1,1)
  plt.subplots_adjust(hspace=0.5)
  uniques = data[feat].nunique()
  print("Processing {}. Unique count: {}.".format(feat, uniques))
  if uniques < 160:
    print(data[feat].unique())
    for feat_value in data[feat].unique():
      if (len(data[data[feat]==feat_value])) / n_items < 0.015:
        print("Adding {} category to OTHER...".format(feat_value))
        data[feat] = data[feat].apply(lambda x: "OTHER" if x==feat_value else x, 1)
  counts = data.groupby(feat).count()
  plt.subplot(len(categorical_features), 1, index_)
  index_ += 1
  counts.iloc[:,0].plot(kind='bar', ax=ax, figsize=(11,12), title='feat', grid=True)
  plt.grid(b=True, which='major', color='b', linestyle='--', axis='y')
  plt.show()
  print(type(counts.iloc[:,0]))
plt.show()

## Feature Engineering

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
categories_le = LabelEncoder()
df.category = categories_le.fit_transform(df.category)


maincategories_le = LabelEncoder()
df.main_category = maincategories_le.fit_transform(df.main_category)


state_le = LabelEncoder()
df.state = state_le.fit_transform(df.state)


country_le = LabelEncoder()
df.country = country_le.fit_transform(df.country)


currency_le = LabelEncoder()
df.currency = currency_le.fit_transform(df.currency)


### Transform / Inverse transform test

In [82]:
currency_le.transform(['CHF'])

array([2])

In [83]:
currency_le.inverse_transform([1])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['CAD'], dtype=object)

### Drop ID, we dont need it now

In [0]:
df = df.drop('ID', axis=1)

In [85]:
df.head()

,blurb,spotlight,staff_pick,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,If you've ever been disheartened by how some p...,True,False,"""How We've Changed Jesus"" - Neal Samudre's Deb...",95,12,13,2013-04-12,3500.0,2013-04-04 22:46:00,3651.0,3,80,21,3651.00,3651.00,3500.00
1,This book will show you how to attract your id...,True,False,Your Client Vision (Book) - How to sell withou...,95,12,5,2017-01-16,175.0,2016-12-18 20:09:14,271.0,3,31,9,94.80,332.73,214.86
2,I told my first lie when I was six and never s...,True,False,The Longest Lie I Ever Told,95,12,1,2016-11-08,3000.0,2016-10-09 18:32:56,3600.0,3,74,3,440.30,2668.45,2223.70
3,Für die Finanzierung der Veröffentlichung mein...,True,False,Spurenlesen – Zur Philosophie der Human-Animal...,95,12,4,2016-08-14,1000.0,2016-07-15 14:17:36,1080.0,3,12,5,222.27,1223.21,1132.61
4,Six teens (age 13-15) are hiking VT's 272-mile...,True,False,Teens Hike VT's Long Trail and Write How-To an...,95,12,13,2014-06-01,3700.0,2014-04-25 02:24:24,4051.2,3,44,21,4051.20,4051.20,3700.00


### Calculate length of funding

In [0]:
diff = (pd.to_datetime(df.deadline) - pd.to_datetime(df.launched)).map(lambda x: x.days)


In [87]:
print(diff.head())

0     7
1    28
2    29
3    29
4    36
dtype: int64


### Drop date times

In [0]:
df = df.drop(['deadline', 'launched'], axis=1)


### Add duration to the dataframe

In [0]:
df['duration'] = diff

In [90]:
df.head()

,blurb,spotlight,staff_pick,name,category,main_category,currency,goal,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,duration
0,If you've ever been disheartened by how some p...,True,False,"""How We've Changed Jesus"" - Neal Samudre's Deb...",95,12,13,3500.0,3651.0,3,80,21,3651.00,3651.00,3500.00,7
1,This book will show you how to attract your id...,True,False,Your Client Vision (Book) - How to sell withou...,95,12,5,175.0,271.0,3,31,9,94.80,332.73,214.86,28
2,I told my first lie when I was six and never s...,True,False,The Longest Lie I Ever Told,95,12,1,3000.0,3600.0,3,74,3,440.30,2668.45,2223.70,29
3,Für die Finanzierung der Veröffentlichung mein...,True,False,Spurenlesen – Zur Philosophie der Human-Animal...,95,12,4,1000.0,1080.0,3,12,5,222.27,1223.21,1132.61,29
4,Six teens (age 13-15) are hiking VT's 272-mile...,True,False,Teens Hike VT's Long Trail and Write How-To an...,95,12,13,3700.0,4051.2,3,44,21,4051.20,4051.20,3700.00,36


TODELETE: Delete columns "Unnamed: 13"-"Unnamed: 16"

In [0]:
#unnecessary_columns = ["Unnamed: {}".format(i) for i in range(13,17)]

In [0]:
#df = df.drop(unnecessary_columns, axis=1)

In [0]:
#df.head()

### Clean text, only keep whitespaces and letters

In [94]:
import re
regex = re.compile('[^a-zA-Z\s]')

name = df.name
name = name.map(lambda x: regex.sub('', str(x)).lower())
print(name.head())

0     how weve changed jesus  neal samudres debut book
1    your client vision book  how to sell without s...
2                          the longest lie i ever told
3    spurenlesen  zur philosophie der humananimalst...
4    teens hike vts long trail and write howto and ...
Name: name, dtype: object


In [0]:
name2 = name.apply(lambda x: x.split())

In [96]:
name2.head()

0    [how, weve, changed, jesus, neal, samudres, de...
1    [your, client, vision, book, how, to, sell, wi...
2                   [the, longest, lie, i, ever, told]
3    [spurenlesen, zur, philosophie, der, humananim...
4    [teens, hike, vts, long, trail, and, write, ho...
Name: name, dtype: object

In [97]:
name.head()

0     how weve changed jesus  neal samudres debut book
1    your client vision book  how to sell without s...
2                          the longest lie i ever told
3    spurenlesen  zur philosophie der humananimalst...
4    teens hike vts long trail and write howto and ...
Name: name, dtype: object

### Remove stop words -> might give better results

In [98]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
s = set(stopwords.words('english'))

In [0]:
name2 = name2.apply(lambda x: [y for y in x if y not in s])

In [101]:
name2.head(10)

0    [weve, changed, jesus, neal, samudres, debut, ...
1       [client, vision, book, sell, without, selling]
2                           [longest, lie, ever, told]
3    [spurenlesen, zur, philosophie, der, humananim...
4    [teens, hike, vts, long, trail, write, howto, ...
5             [ako, ay, pilipino, coffee, table, book]
6                                        [truth, book]
7    [forks, three, years, five, continents, one, m...
8                                [mysterious, letters]
9          [motorcycle, adventures, farlap, max, book]
Name: name, dtype: object

### Create bag of words
Using HashingVectorizer, because there are a lot of words

In [102]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features=2**8, lowercase=False, tokenizer = lambda doc: doc)
hashed_name = vectorizer.transform(name2)
print(hashed_name.shape)
print(hashed_name[0])

(160317, 256)
  (0, 14)	0.3779644730092272
  (0, 24)	0.3779644730092272
  (0, 89)	0.3779644730092272
  (0, 96)	-0.3779644730092272
  (0, 132)	-0.3779644730092272
  (0, 159)	-0.3779644730092272
  (0, 161)	-0.3779644730092272


In [103]:
print(hashed_name[0])

  (0, 14)	0.3779644730092272
  (0, 24)	0.3779644730092272
  (0, 89)	0.3779644730092272
  (0, 96)	-0.3779644730092272
  (0, 132)	-0.3779644730092272
  (0, 159)	-0.3779644730092272
  (0, 161)	-0.3779644730092272


In [104]:
print(type(hashed_name))

<class 'scipy.sparse.csr.csr_matrix'>


In [105]:
print(hashed_name.todense().shape)

(160317, 256)


### Create dataframe from scipy sparse matrix

In [0]:
df_hashed_name = pd.DataFrame(hashed_name.todense())

In [107]:
df_hashed_name.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,-0.333333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Concat original matrix and hashed name matrix

In [0]:
df = df.join(df_hashed_name)

In [109]:
print(df.shape)

(160317, 272)


### Dropping name column, name is stored now in bag

In [0]:
df = df.drop('name', axis=1)

### Split into input and output

In [0]:
input_columns = ['category', 'main_category', 'currency', 'goal', 'duration', 'country']
output_columns = ['state', 'backers', 'usd pledged']

X = df.drop(output_columns, axis=1)
Y = df[df.columns.intersection(output_columns)]

In [112]:
print(X.head())

print()

print(Y.head())

                                               blurb  spotlight  staff_pick  \
0  If you've ever been disheartened by how some p...       True       False   
1  This book will show you how to attract your id...       True       False   
2  I told my first lie when I was six and never s...       True       False   
3  Für die Finanzierung der Veröffentlichung mein...       True       False   
4  Six teens (age 13-15) are hiking VT's 272-mile...       True       False   

   category  main_category  currency    goal  pledged  country  \
0        95             12        13  3500.0   3651.0       21   
1        95             12         5   175.0    271.0        9   
2        95             12         1  3000.0   3600.0        3   
3        95             12         4  1000.0   1080.0        5   
4        95             12        13  3700.0   4051.2       21   

   usd_pledged_real ...   246  247  248  249  250  251  252  253  254  255  
0           3651.00 ...   0.0  0.0  0.0  0.0  0.0  

## Preparation for teaching - DONT RUN YET

### Imports

In [53]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
import copy
import pandas as pd
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
from tensorflow import set_random_seed
set_random_seed(123)
np.random.seed(123)

Using TensorFlow backend.


### Splitting into train test valid

In [54]:
test_split = 0.1
valid_split = 0.2

v_index = int(X.shape[0] * (1-valid_split-test_split))
t_index = int(X.shape[0] * (1-test_split))

X_test = X[t_index:]
Y_test = Y[t_index:]

X_valid = X[v_index:t_index]
Y_valid = Y[v_index:t_index]
X_train = X[:v_index]
Y_train = Y[:v_index]

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

ValueError: ignored

### Creating model

In [293]:
es = EarlyStopping(patience=10)
mcp = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)

model = Sequential()
model.add(Dense(276, input_shape=(X_train.shape[1],)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(Y_train.shape[1], activation='linear'))

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=sgd)

history = model.fit(X_train, Y_train, epochs=100000, batch_size=256,
                    verbose=2, validation_data=(X_valid,Y_valid),
                   callbacks=[mcp, es])

Train on 208761 samples, validate on 59646 samples
Epoch 1/100000
 - 7s - loss: 1525003996.1892 - val_loss: 2113103641.6656

Epoch 00001: val_loss improved from inf to 2113103641.66556, saving model to weights.hdf5
Epoch 2/100000
 - 7s - loss: 1495636853.5889 - val_loss: 2089401925.3510

Epoch 00002: val_loss improved from 2113103641.66556 to 2089401925.35104, saving model to weights.hdf5
Epoch 3/100000
 - 7s - loss: 1482824073.5110 - val_loss: 2072122294.0123

Epoch 00003: val_loss improved from 2089401925.35104 to 2072122294.01227, saving model to weights.hdf5
Epoch 4/100000
 - 7s - loss: 1478227224.7791 - val_loss: 2058185866.4682

Epoch 00004: val_loss improved from 2072122294.01227 to 2058185866.46816, saving model to weights.hdf5
Epoch 5/100000
 - 7s - loss: 1466802995.4491 - val_loss: 2060145036.1560

Epoch 00005: val_loss did not improve from 2058185866.46816
Epoch 6/100000
 - 7s - loss: 1463354981.1764 - val_loss: 2045504113.8112

Epoch 00006: val_loss improved from 2058185866

In [0]:
from keras.models import load_model

model = load_model('weights.hdf5')

In [295]:
from sklearn.metrics import mean_squared_error
preds = model.predict(X_test)
err = mean_squared_error(Y_test, preds)

print("Error on test data: {}".format(err))

Error on test data: 1202461271.5459595


In [296]:
print(preds[0][:4])
print(type(preds))
print(type(Y_test))

[1.8545226e+00 1.4313721e+02 1.2102148e+04]
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [297]:
print(preds.shape)
print(Y_test.shape)

(29824, 3)
(29824, 3)


In [0]:
pred_df = pd.DataFrame(data=preds)

In [299]:
pred_df.head()

,0,1,2
0,1.854523,143.137207,12102.148438
1,1.234310,45.287964,3239.476562
2,1.476785,-86.171631,-9614.062500
3,1.154954,-172.237183,-14926.867188
4,1.768590,-140.093506,-12001.695312


In [302]:
Y_test.head()

,state,backers,usd pledged
268407,successful,121,8620.0
268408,failed,0,0.0
268409,successful,62,3406.0
268410,failed,4,86.0
268411,canceled,1,54.0


In [301]:
Y_test.state = state_le.inverse_transform(Y_test.state)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [303]:
pred_df[0]

0        1.854523
1        1.234310
2        1.476785
3        1.154954
4        1.768590
5        2.020177
6        1.712959
7        1.137712
8        1.637320
9        1.431555
10       2.144732
11       1.905660
12       2.539652
13       1.849550
14       1.737180
15       0.628774
16       1.662036
17       1.850657
18       1.454541
19       1.595048
20       1.125447
21       1.566085
22       1.661106
23       1.392544
24       1.825468
25       1.715683
26       0.966332
27       1.367498
28       1.831913
29       3.065158
           ...   
29794    1.367898
29795    0.809116
29796    1.963822
29797    1.597848
29798    1.696616
29799    1.838276
29800    1.561378
29801    1.568236
29802    1.906662
29803    1.389450
29804    1.700372
29805    1.298922
29806    1.398926
29807    1.357550
29808    1.213487
29809    1.825180
29810    1.381535
29811    1.473561
29812    1.693005
29813    1.475507
29814    1.516240
29815    1.467871
29816    1.952759
29817    1.431612
29818    0

In [0]:
pred_df[0] = pred_df[0].apply(lambda x: int(np.round(x)) if x >= 0 else 0)

In [310]:
pred_df

,0,1,2
0,2,143.137207,12102.148438
1,1,45.287964,3239.476562
2,1,-86.171631,-9614.062500
3,1,-172.237183,-14926.867188
4,2,-140.093506,-12001.695312
5,2,127.593018,8610.632812
6,2,86.509155,7090.281250
7,1,-123.127991,-10050.539062
8,2,294.510498,20045.093750
9,1,-104.020752,-10256.656250


In [311]:
pred_df[0] = state_le.inverse_transform(pred_df[0])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [314]:
print(pred_df.head())
print(Y_test.head())

        0           1             2
0    live  143.137207  12102.148438
1  failed   45.287964   3239.476562
2  failed  -86.171631  -9614.062500
3  failed -172.237183 -14926.867188
4    live -140.093506 -12001.695312
             state  backers  usd pledged
268407  successful      121       8620.0
268408      failed        0          0.0
268409  successful       62       3406.0
268410      failed        4         86.0
268411    canceled        1         54.0
